In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('/kaggle/input/predict-test-scores-of-students/test_scores.csv')
data.drop(['classroom', 'student_id'], axis = 1, inplace = True)
data.head()

In [ ]:
data.info()

## Descriptive Statistics

In [ ]:
data.describe().round(2)

In [ ]:
fig, ax = plt.subplots(figsize = (20, 6))
ax.set_title("Posttest Scores", fontsize = 20)
ax.set_ylabel("Scores", fontsize = 16)
ax.set_xlabel("Student", fontsize = 16)
ax = data['posttest'].plot(fontsize = 14)

In [ ]:
def plot_boxplot(title, xlabel, ylabel, x, y, dataset, rotation=0):
    ax = sns.boxplot(data=dataset, x=x, y=y, orient='v', width = 0.2)
    ax.figure.set_size_inches(12, 6)
    ax.set_title(title, fontsize = 20)
    ax.set_ylabel(ylabel, fontsize = 16)
    ax.set_xlabel(xlabel, fontsize = 16)
    ax.set_xticklabels(ax.get_xticklabels(),rotation=rotation)
    ax

In [ ]:
def plot_pairplot(title, x, y, dataset, hue=None):
    ax = sns.pairplot(dataset, y_vars = y, x_vars = x, hue=hue)
    ax.fig.suptitle(title, fontsize = 20, y=1)
    ax.fig.set_size_inches(20,12)
    if hue:
        ax._legend.remove()
    plt.legend(bbox_to_anchor=(1.01, 1), borderaxespad=0)

In [ ]:
plot_boxplot("Boxplot of Posttest Scores", '', "Scores", 'posttest', None, data)

In [ ]:
plot_boxplot("Boxplot of Posttest Scores by School", "School", "Scores", 'school', 'posttest', data, 30)

In [ ]:
plot_boxplot("Boxplot of Posttest Scores by School Setting", "School Setting", "Scores", 'school_setting', 'posttest', data)

In [ ]:
plot_boxplot("Boxplot of Posttest Scores by School Type", "School Type", "Scores", 'school_type', 'posttest', data)

In [ ]:
plot_boxplot("Boxplot of Posttest Scores by Teaching Method", "Teaching Method", "Scores", 'teaching_method', 'posttest', data)

In [ ]:
plot_boxplot("Boxplot of Posttest by whether a student qualifies for free/subsidized lunch", "Lunch", "Scores", 'lunch', 'posttest', data)

In [ ]:
sns.histplot(x = data['posttest'], kde = True)

In [ ]:
plot_pairplot("Distribution of Posttest Scores", ['pretest'], 'posttest', data)

In [ ]:
plot_pairplot("Distribution of Posttest Scores by School Setting", ['pretest'], 'posttest', data, 'school_setting')

In [ ]:
plot_pairplot("Distribution of Posttest Scores by School Type", ['pretest'], 'posttest', data, 'school_type')

In [ ]:
plot_pairplot("Distribution of Posttest Scores by Teaching Method", ['pretest'], 'posttest', data, 'teaching_method')

In [ ]:
plot_pairplot("Distribution of Posttest Scores by whether a student qualifies for free/subsidized lunch", ['pretest'], 'posttest', data, 'lunch')

Looking at the boxplots you can see that there are some characteristics that affect the posttest score of the students.

There are big differences in the scores from school to school, with some clearly performing better than others.


#### On average*:

The School Setting with higher scores is "Suburban"

The School Type with higher scores is "Non-public"

The Teaching Method with higher scores is "Experimental"

And students that does not qualify free/subsidized lunch have higher scores


*This is just a visual analysis, it does not have a statistical proof

## Model

In [ ]:
# split X and Y
X = data.drop('posttest', axis = 1)
y = data['posttest']

In [ ]:
# One_Hot encode

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

categories = ['school', 'school_setting', 'school_type', 'teaching_method', 'gender', 'lunch']

one_hot = OneHotEncoder()
transformer = ColumnTransformer([('one_hot', one_hot, categories)],
                                remainder = 'passthrough')
X_transformed = transformer.fit_transform(X)

In [ ]:
# Split the data into train and test
np.random.seed(9)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size = 0.2)

In [ ]:
# Fit the model

from xgboost import XGBRegressor

model = XGBRegressor(n_estimators=1000, learning_rate=0.1, n_jobs=4)

model.fit(X_train, y_train, 
          early_stopping_rounds=10, 
          eval_set=[(X_test, y_test)],
          verbose=False)

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

preds = model.predict(X_test)

r2 = r2_score(y_test, preds)
mae = mean_absolute_error(y_test, preds)
mse = mean_squared_error(y_test, preds)

print(f'R2_score: {r2 * 100:.2f}%')
print(f'MAE: {mae:.2f}')